## 111 Aggregating Data using Group By in Spark SQL

In [1]:
val username = System.getProperty("user.name")

username = itversity


itversity

In [2]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Managing Tables - Basic DDL and DML2").
    master("yarn").
    getOrCreate

username = itversity
spark = org.apache.spark.sql.SparkSession@7e67777


org.apache.spark.sql.SparkSession@7e67777

In [3]:
%%sql

use itversity_retail

Waiting for a Spark session to start...

++
||
++
++



In [4]:
%%sql 

show tables

+----------------+-----------+-----------+
|        database|  tableName|isTemporary|
+----------------+-----------+-----------+
|itversity_retail|order_items|      false|
|itversity_retail|     orders|      false|
|itversity_retail|   students|      false|
+----------------+-----------+-----------+



In [5]:
%%SQL

select count(1) from orders

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [7]:
%%sql

select count(distinct order_date) from orders

+--------------------------+
|count(DISTINCT order_date)|
+--------------------------+
|                       364|
+--------------------------+



In [10]:
%%sql

select round(sum(order_item_subtotal),2) from order_items 
where order_item_order_id=2

+--------------------------------------------------+
|round(sum(CAST(order_item_subtotal AS DOUBLE)), 2)|
+--------------------------------------------------+
|                                            579.98|
+--------------------------------------------------+



In [20]:
%%sql

select count(1)
from orders
where order_status in ('COMPLETE','CLOSED')

+--------+
|count(1)|
+--------+
|   30455|
+--------+



In [12]:
%%SQL

SELECT order_date, count(1)
from orders
group by order_date

+--------------------+--------+
|          order_date|count(1)|
+--------------------+--------+
|2013-08-13 00:00:...|      73|
|2014-03-19 00:00:...|     130|
|2014-04-26 00:00:...|     251|
|2013-10-12 00:00:...|     162|
|2013-11-15 00:00:...|     135|
|2013-09-16 00:00:...|     121|
|2013-09-20 00:00:...|     139|
|2013-12-31 00:00:...|     266|
|2013-09-06 00:00:...|     276|
|2014-06-15 00:00:...|     128|
+--------------------+--------+
only showing top 10 rows



In [13]:
%%sql

select order_status, count(1) from orders group by order_status

+---------------+--------+
|   order_status|count(1)|
+---------------+--------+
|PENDING_PAYMENT|   15030|
|       COMPLETE|   22899|
|        ON_HOLD|    3798|
| PAYMENT_REVIEW|     729|
|     PROCESSING|    8275|
|         CLOSED|    7556|
|SUSPECTED_FRAUD|    1558|
|        PENDING|    7610|
|       CANCELED|    1428|
+---------------+--------+



In [15]:
%%sql

select order_item_order_id, round(sum(order_item_subtotal),2) as order_revenue
from order_items
group by order_item_order_id limit 10

+-------------------+-------------+
|order_item_order_id|order_revenue|
+-------------------+-------------+
|                148|       479.99|
|                463|       829.92|
|                471|       169.98|
|                496|       441.95|
|               1088|       249.97|
|               1580|       299.95|
|               1591|       439.86|
|               1645|      1509.79|
|               2366|       299.97|
|               2659|       724.91|
+-------------------+-------------+



In [16]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id, round(sum(oi.order_item_subtotal),2) as revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
LIMIT 10

+--------------------+------...


+--------------------+---------------------+-------+
|          order_date|order_item_product_id|revenue|
+--------------------+---------------------+-------+
|2013-07-27 00:00:...|                  703|  39.98|
|2013-07-29 00:00:...|                  793|  44.97|
|2013-08-12 00:00:...|                  627| 3199.2|
|2013-08-15 00:00:...|                  926|  15.99|
|2013-09-04 00:00:...|                  957|3599.76|
|2013-09-07 00:00:...|                  235| 104.97|
|2013-09-17 00:00:...|                  792|  14.99|
|2013-09-25 00:00:...|                   44| 239.96|
|2013-09-27 00:00:...|                  276|  31.99|
|2013-10-04 00:00:...|                  792|  44.97|
+--------------------+---------------------+-------+



In [17]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id, round(sum(oi.order_item_subtotal),2) as revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
    AND revenue >=500
GROUP BY o.order_date,
    oi.order_item_product_id
LIMIT 10

            :  +- SubqueryAlias...


Magic sql failed to execute with error: 
cannot resolve '`revenue`' given input columns: [oi.order_item_quantity, oi.order_item_product_id, oi.order_item_product_price, oi.order_item_id, o.order_id, o.order_date, o.order_customer_id, oi.order_item_subtotal, o.order_status, oi.order_item_order_id]; line 1 pos 231;
'GlobalLimit 10
+- 'LocalLimit 10
   +- 'Aggregate ['o.order_date, 'oi.order_item_product_id], ['o.order_date, 'oi.order_item_product_id, 'round('sum('oi.order_item_subtotal), 2) AS revenue#183]
      +- 'Filter (order_status#187 IN (COMPLETE,CLOSED) && ('revenue >= 500))
         +- Join Inner, (order_id#184 = order_item_order_id#189)
            :- SubqueryAlias `o`
            :  +- SubqueryAlias `itversity_retail`.`orders`
            :     +- HiveTableRelation `itversity_retail`.`orders`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [order_id#184, order_date#185, order_customer_id#186, order_status#187]
            +- SubqueryAlias `oi`
               +- SubqueryAl

In [22]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id, round(sum(oi.order_item_subtotal),2) as revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
    AND round(sum(oi.order_item_subtotal),2) >=500
GROUP BY o.order_date,
    oi.order_item_product_id
LIMIT 10

               +- *(2) Broad...


Magic sql failed to execute with error: 
execute, tree:
Exchange SinglePartition
+- *(3) LocalLimit 10
   +- *(3) HashAggregate(keys=[order_date#294, order_item_product_id#299], functions=[sum(cast(order_item_subtotal#301 as double))], output=[order_date#294, order_item_product_id#299, revenue#292])
      +- Exchange hashpartitioning(order_date#294, order_item_product_id#299, 200)
         +- *(2) HashAggregate(keys=[order_date#294, order_item_product_id#299], functions=[partial_sum(cast(order_item_subtotal#301 as double))], output=[order_date#294, order_item_product_id#299, sum#318])
            +- *(2) Project [order_date#294, order_item_product_id#299, order_item_subtotal#301]
               +- *(2) BroadcastHashJoin [order_id#293], [order_item_order_id#298], Inner, BuildLeft
                  :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
                  :  +- *(1) Project [order_id#293, order_date#294]
                  :     +- *(1) 

In [18]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id, round(sum(oi.order_item_subtotal),2) as revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
HAVING revenue >=500
LIMIT 10

+--------------------+------...


+--------------------+---------------------+-------+
|          order_date|order_item_product_id|revenue|
+--------------------+---------------------+-------+
|2013-08-04 00:00:...|                  403|3119.76|
|2013-09-08 00:00:...|                 1073|3199.84|
|2013-09-09 00:00:...|                  957|5399.64|
|2013-09-15 00:00:...|                  627|1039.74|
|2013-09-22 00:00:...|                  365|6058.99|
|2013-10-03 00:00:...|                  565|  700.0|
|2013-11-03 00:00:...|                  957|8099.46|
|2013-11-25 00:00:...|                 1014|2798.88|
|2013-12-19 00:00:...|                  191|2399.76|
|2013-12-22 00:00:...|                  403|5069.61|
+--------------------+---------------------+-------+

